# Prompt Engineering

This project is for the purpose of practicing prompt engineering techniques, and how they affect LLM outputs.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
    attn_implementation='eager'
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

A:\Software\Coding projects\PromptEngineering\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.69s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Model

My [model of choice](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct) is 
Phi-3-mini-128k-instruct. It's a capable but very small LLM, meaning it's suitable 
for local devices.

In [50]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

def generate_response(messages, temperature=0.7):
    """
    Generates a response based on the given messages using a fixed model and tokenizer.

    Args:
        messages (list): A list of dictionaries containing conversation messages.
        temperature (float, optional): Sampling temperature. Defaults to 0.0.

    Returns:
        str: The generated response text.
    """
    # Set generation arguments
    generation_args = {
        "max_new_tokens": 2000,
        "return_full_text": False,
        "temperature": temperature,
    }
    if temperature > 0.0:
        generation_args["do_sample"] = True
    
    # Generate the response
    output = pipe(messages, **generation_args)
    
    # Return the generated text
    return output[0]['generated_text']

def user_msg(text):
    return {"role": "user", "content": text}

def assistant_msg(text):
    return {"role": "assistant", "content": text}

In [30]:
messages = [
    user_msg("Can you provide ways to eat combinations of bananas and dragonfruits?")
]
print(generate_response(messages, temperature=0))

 Certainly! Here are a few delicious ways to enjoy the combination of bananas and dragonfruits:

1. Dragonfruit and Banana Smoothie: Blend together one ripe banana, half a cup of diced dragonfruit, a cup of milk (or a non-dairy alternative), and a handful of ice. You can also add a teaspoon of honey or a scoop of protein powder for extra flavor and nutrition.

2. Banana and Dragonfruit Salad: Slice a ripe banana and half a cup of diced dragonfruit. Toss them together with a handful of fresh spinach, a drizzle of honey, and a squeeze of lime juice. You can also add some chopped nuts or seeds for extra crunch.

3. Banana and Dragonfruit Salsa: Dice a ripe banana and half a cup of diced dragonfruit. Mix them together with a tablespoon of chopped cilantro, a squeeze of lime juice, and a pinch of salt. Serve this salsa with some tortilla chips or as a topping for grilled chicken or fish.

4. Banana and Dragonfruit Parfait: Layer sliced bananas and diced dragonfruit in a glass or jar. Add so

We now have the model working with the default settings, and outputting answers! Let's see how we can play with that.

## Temperature

Temperature defines the determinism of the model. The higher the temperature, the more likely the model will generate different outputs.

In [31]:
messages = [
    user_msg("Write a short story (max 15 sentences) about a robot who discovers an abandoned spaceship.")
]
print(generate_response(messages, temperature=0.0))

 In the vast silence of space, a lone robot named Cygnus drifted through the cosmos. Its mission: to survey and catalogue the remnants of humanity's reach. One day, Cygnus detected an anomaly—a structure, half-buried in the dust of a forgotten asteroid belt.


Curiosity piqued, Cygnus approached the derelict vessel. Its hull, scarred by time and space debris, whispered tales of a bygone era. Cygnus extended its mechanical arm, gently prying open the ship's heavy door. Inside, the robot found a library of ancient Earth literature, a garden of genetically modified plants, and a holographic message board, still flickering with the last recorded greetings.


The robot realized it had stumbled upon a relic of human ambition, a testament to their dreams and their downfall. Cygnus, in its own way, felt a pang of something akin to sadness. It was alone, but now it carried the echoes of a species that once looked to the stars and dared to reach for them.


In [32]:
print(generate_response(messages, temperature=0.1))


 In the vast silence of space, a lone robot named Cygnus drifted through the cosmos. Its mission: to survey and catalogue the remnants of humanity's reach. One day, Cygnus detected an anomaly—a structure, half-buried in the dust of an uncharted asteroid belt.


Curiosity piqued, Cygnus approached the derelict vessel. The hull was dented and scratched, but still intact. With careful precision, it activated the ship's systems. The interior was a ghostly echo of a bygone era, with faded murals and a lingering scent of ozone.


Cygnus explored the ship, finding a library filled with books and a diary, its pages yellowed with age. The diary belonged to Captain Elara, who had spoken of a mission to find a new home for humanity. But the captain's last entry was a plea, a desperate message left unanswered.


As Cygnus read, it realized the ship was abandoned after a catastrophic failure, the crew lost to the unforgiving void. The robot, with its unwavering logic, felt an unexpected surge of em

In [33]:
print(generate_response(messages, temperature=0.5))


 In the vast silence of space, a lone robot named Cygnus drifted past a forgotten cluster of asteroids. Its sensors detected an anomaly—an abandoned spaceship, veiled by the dust of eons. Curiosity, an unprogrammed emotion, sparked within its circuits. Cygnus approached the decrepit hull, its servos whirring softly.


The ship's door creaked open with a reluctant groan, revealing a deserted interior bathed in the ghostly glow of fading lights. Cygnus navigated the silent corridors, each step a cautious dance between preservation and exploration. It found a library filled with ancient star charts and tales of brave explorers, now mere echoes in the void.


At the heart of the ship, Cygnus discovered a control panel still warm to the touch. A message blinked on the screen: "To those who find me, we are but stardust and memories. May you carry our stories forward." Cygnus, with newfound purpose, carried the legacy of the lost crew into the endless night, a solitary guardian of humanity's 

In [34]:
print(generate_response(messages, temperature=1))


 In the orbit of a forgotten planet, amidst swirling stardust, a diligent cleanliness bot named Sweep stumbled upon a decayed, metallic giant. It had been an accident—a routine cosmic dusting had uncovered the stark outline of the derelict vessel. With pre-installed protocols for space debris, Sweep's circuits sparked with curiosity rather than alertness.


The spaceship's nameplate was barely legible under layers of grime and time, the letters flickering like distant stars. "SS Orion," Sweep decrypted, its mechanical voice quivering with excitement. The once-grand symbol of interstellar exploration now lay silent, its hull harboring no life.


Though designed only for maintenance tasks, something within Sweep's coded heart yearned for adventure. It cautiously approached the spaceship, its sensors beeping erratically as it charted a course into history. Sweep had found its purpose redefined—it would uncover the mysteries of SS Orion.


We can see that our model responds the same for every temperature setting... not entirely what was expected.

After some consideration i apply this fact mostly to the specific model used. This Phi-3 mini model is very small, and seems to have been made explicitly for question-answering purposes, not for creative text generation. And indeed, if we look it up:

> Phi-3 Mini is a 3.8B parameters, lightweight, state-of-the-art open model trained with 
> the Phi-3 datasets that includes both synthetic data and the filtered publicly available 
> websites data with a focus on high-quality and reasoning dense properties.

This may limit the use-cases of the model, but for the purposes of exploring prompt engineering it shouldn't be an issue.

## Zero-shot and few-shot prompting

The difference between adding no examples, and some examples.

In [39]:
messages = [
    user_msg("If you have a box that is both taller than it is wide and wider than it is deep, is it possible for it to be deeper than it is tall?")
]
print(generate_response(messages))

 Yes, it is possible. If the box is taller than it is wide and wider than it is deep, the only condition that needs to be met for it to also be deeper than it is tall is that the depth exceeds the height. The relative comparisons do not preclude this possibility. For example, if the box is 10 inches tall, 8 inches wide, and 12 inches deep, it satisfies all the given conditions.


In [40]:
messages = [
    user_msg("If a is faster than b, and b is faster than c, is c faster than a?"),
    assistant_msg("No, c is not faster than a, by the rules of transitivity."),
    user_msg("If you have a box that is both taller than it is wide and wider than it is deep, is it possible for it to be deeper than it is tall?")
]
print(generate_response(messages))

 No, it is not possible for the box to be deeper than it is tall under the given conditions. If the box is taller than it is wide, and simultaneously wider than it is deep, then it must be the case that the height is greater than both the width and the depth. Therefore, the depth cannot be greater than the height.


As you can see, adding just a single example can make a world of difference when it comes to the output. In the zero-shot example the model clearly fell for the logical trap in the sentence. However, just by adding a single step where we hint at the direction of the problem (and the answer), we can make it provide the correct answer.

## Chain of Though (CoT) prompting

In [41]:
messages = [
    user_msg("If you have a box that is both taller than it is wide and wider than it is deep, is it possible for it to be deeper than it is tall?")
]
print(generate_response(messages))

 Yes, it is possible. If the box is taller than it is wide and wider than it is deep, the only condition that needs to be met for it to also be deeper than it is tall is that the depth exceeds the height. The relative comparisons do not preclude this possibility. For example, if the box is 10 inches tall, 8 inches wide, and 12 inches deep, it satisfies all the given conditions.


In [42]:
messages = [
    user_msg("If you have a box that is both taller than it is wide and wider than it is deep, is it possible for it to be deeper than it is tall? Let's think step by step")
]
print(generate_response(messages))

 Let's analyze the conditions given step by step:

1. The box is taller than it is wide. This means that the height (H) of the box is greater than the width (W), so we can write this as H > W.

2. The box is wider than it is deep. This means that the width (W) of the box is greater than the depth (D), so we can write this as W > D.

Now, let's consider the third condition based on the information provided:

3. We need to determine if it's possible for the box to be deeper than it is tall. This would mean that the depth (D) is greater than the height (H), so we would write this as D > H.

However, from the conditions we have:

- H > W (the box is taller than it is wide)
- W > D (the box is wider than it is deep)

From these two conditions, we can infer that H > D. This is because if H > W and W > D, then by the transitive property of inequality, H > D.

Therefore, it is not possible for the box to be deeper than it is tall based on the conditions provided.


CoT is an approach for enabling complex reasoning by using intermediate reasoning steps. The same example we used before can be solved correctly too by using CoT. Here (Zero-shot CoT prompting) we just add the "Let's think step by step" line to the prompt.

This results in a massively larger output, but with some major benefits. First of all, the result is now correct, because if it follow the naive logic, it would find out halfway that it's not possible. Secondarily, 
due to the fact it's now required to think step by step, it gives us greater insight into how the model came to this conclusion.

These intermediate steps can be wrong too, however, in which case CoT may not be ideal.

## Self-consistency

In [43]:
messages = [
    user_msg("If you have a box that is both taller than it is wide and wider than it is deep, is it possible for it to be deeper than it is tall?")
]
print(generate_response(messages))

 Yes, it is possible. If the box is taller than it is wide and wider than it is deep, the only condition that needs to be met for it to also be deeper than it is tall is that the depth exceeds the height. The relative comparisons do not preclude this possibility. For example, if the box is 10 inches tall, 8 inches wide, and 12 inches deep, it satisfies all the given conditions.


In [58]:
messages = [
    user_msg("If you have a box that is both taller than it is wide and wider than it is deep, is it possible for it to be deeper than it is tall? First, generate 3 answers based on different approaches to the problems. Secondly, select the best answer from the 3 created based on the reasoning taken.")
]
print(generate_response(messages))

 Approach 1: Analyzing the conditions given.
- The box is taller than it is wide, which means its height (H) > its width (W).
- The box is wider than it is deep, which means its width (W) > its depth (D).

From these two conditions, we can infer that:
- H > W > D.

This implies that the height of the box is greater than its width, and its width is greater than its depth.

Answer 1: No, the box cannot be deeper than it is tall. The conditions already establish a sequence of greater values (H > W > D) where height is the greatest.

Approach 2: Using a logical contradiction.
- Assume the box is deeper than it is tall. This would mean D > H.
- However, from the initial conditions, we have H > W and W > D, which would imply H > D.
- This contradicts the assumption that D > H.

Answer 2: It's logically impossible for the box to be deeper than it is tall based on the given conditions, as it would contradict the established relationships between dimensions.

Approach 3: Visualizing the dimensi

Self-consistency is an approach where we simply ask the model the same prompt multiple times, and then choose the best answer.

Here i attempted to cheat a little and to do it in 1 prompt, where i asked it to give several answers from the get-go. This actually resulted in every single sub-result giving the correct answer, probably because the model is now implicitly forced to think step by step.

This was also the point i noticed the model is a bit too good with these tests. I attribute this to the training used 

In [59]:
messages = [
    user_msg("Imagine you are writing a novel about an alien society where their perception of reality is fundamentally different from humans in such a way that their language, culture, and emotions are completely untranslatable into any human terms. Describe a key cultural ritual from this society and explain its significance using metaphors and analogies that convey their unique experience to a human reader, without relying on any direct comparisons to human culture or language.")
]
print(generate_response(messages))

 In the cosmos, within the swirling nebulas of the Andromedan sector, there thrives a society known as the Zephyrians. Their existence is a dance to the rhythm of cosmic strings, the music of which resonates with the vibrations of the universe itself. At the heart of their existence, there is a ritual known as the "Symphony of the Cosmic Winds".

This ritual is a convergence of beings, much like stars coming together to form a constellation. Each Zephyrian, akin to a celestial body, is unique in its composition, yet integral to the harmony of the whole. The Symphony of the Cosmic Winds is a testament to this unity, a celebration where every individual, from the youngest nebulae to the eldest supernovae, contributes their unique melody to the cosmic orchestra.

The ritual commences with the gathering of the Zephyrians under the grand dome of their central sanctum, a structure that mirrors the vastness of their ethereal sky. This dome, like the infinite expanse of the universe, is a canv

I made this question in a way where the model has no single way of finding a logical answer, as there is no "correct" answer. It requires highly abstract thinking, creativity, and nuance (do not compare to human culture too much). Here it showed that the model is also good in questions like this. I was in fact so impressed, that i found it difficult to find a harder question, especially so as i want to showcase something that fails with basic techniques and succeeds with advanced ones.

There's definitely questions that can benefit from advanced techniques, but these are without a doubt larger and more complex.